In [1]:
# Darion Pescada

## Research question/interests

How have the prices of games changed over the years?


In [3]:
#Milestone 3
#Task 1

import pandas as pd 
import numpy as np
import seaborn as sns

dfr = pd.read_csv('../data/raw/steam.csv')

dfp = dfr.copy().drop(['name', 'genres','appid', 'english', 'developer', 'publisher', 'required_age', 'platforms', 'achievements', 'categories', 
'steamspy_tags', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners'], axis=1)

print("Shape: ", dfp.shape)

display(dfp.head())

display(dfp.describe())

print("year: game_count:")
dfp['release_date'] = [int(i.split('-')[0]) for i in dfp['release_date']]
dfp['release_date'].value_counts()


Shape:  (27075, 2)


,release_date,price
0,2000-11-01,7.19
1,1999-04-01,3.99
2,2003-05-01,3.99
3,2001-06-01,3.99
4,1999-11-01,3.99


,price
count,27075.000000
mean,6.078193
std,7.874922
min,0.000000
25%,1.690000
50%,3.990000
75%,7.190000
max,421.990000


year: game_count:


2018    8160
2017    6357
2016    4361
2015    2597
2019    2213
2014    1555
2013     418
2012     320
2009     305
2011     239
2010     238
2008     145
2007      93
2006      48
2005       6
2004       6
2001       4
2003       3
2000       2
1999       2
2002       1
1998       1
1997       1
Name: release_date, dtype: int64

The max price is unreasonably high and the min is zero which could greatly influence the outcome of the data such as the average price so these games should not be included in the analysis.  
There is insufficient data from before 2006 that should also not be accounted for.   
The amount of games released each year increases dramatically after 2005 which may affect the average prices.  

In [5]:
#Task 2

import pandas as pd 
import numpy as np
import seaborn as sns

#load data
dfr = pd.read_csv('../data/raw/steam.csv')

#drop irrelevant columns
dfp = dfr.copy().drop(['name', 'genres', 'appid', 'english', 'developer', 'publisher', 'required_age', 'platforms', 'achievements', 'categories', 
'steamspy_tags', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners'], axis=1)

#remove games with a price of 0 or above 100 as I don't want unreasonably expensive games and free games included in the analysis
dfp = dfp.drop(dfp[dfp.price > 100].index).reset_index(drop=True)
dfp = dfp.drop(dfp[dfp.price == 0].index).reset_index(drop=True)

#drop NaN rows if any
dfp = dfp.dropna(axis=0).reset_index(drop=True)

#drop the month and day of release date, only keepinig the year
dfp['release_date'] = [int(i.split('-')[0]) for i in dfp['release_date']]
#sort rows by year
dfp = dfp.sort_values(by='release_date', ascending=True)


#dropping data from before 2006 because it is insufficient
#made 2006 a variable incase I end up changing the starting year
starting_year = 2006
dfp = dfp.drop(dfp[dfp.release_date < starting_year].index)
dfp = dfp.reset_index()

#the code below creates lists of the years, the amount of games, the sum of game prices and the average prices
#it is messy and probably not the best way to do this but it works
years = []
added_price = []
game_count = []
average_price = []
    
for x in range(2020 - starting_year):   
    years.append(starting_year + x)
    added_price.append(0)
    game_count.append(0)
    average_price.append(0)
    for y in range(len(dfp)):
        if dfp['release_date'][y] == x + starting_year:
            added_price[x] = added_price[x] + dfp['price'][y]
            game_count[x] = game_count[x] + 1
    if game_count[x] != 0:
        average_price[x] = added_price[x] / game_count[x]

#create a dictionary of the new lists
d = {"Year": years, "Game_Count": game_count, "Price_Sum": added_price, "Price_Average": average_price}

#create dataframe using dictionary
dfn = pd.DataFrame(d)

display(dfp)
display(dfn)
   


,index,release_date,price
0,141,2006,3.99
1,56,2006,14.99
2,57,2006,14.99
3,58,2006,14.99
4,62,2006,4.99
...,...,...,...
24479,13635,2019,19.49
24480,19407,2019,3.99
24481,23015,2019,3.99
24482,23060,2019,3.99


,Year,Game_Count,Price_Sum,Price_Average
0,2006,41,362.81,8.849024
1,2007,87,523.87,6.021494
2,2008,137,947.61,6.916861
3,2009,295,2132.07,7.227356
4,2010,230,1758.55,7.645870
5,2011,223,1799.15,8.067937
6,2012,296,2748.34,9.284932
7,2013,388,3634.40,9.367010
8,2014,1473,11604.65,7.878242
9,2015,2409,16670.33,6.920021


In [6]:
#Task 3

import project_functions2
df = project_functions2.load_and_process("../data/raw/steam.csv")
df

,Year,Game_Count,Price_Sum,Price_Average
0,2006,41,362.81,8.849024
1,2007,87,523.87,6.021494
2,2008,137,947.61,6.916861
3,2009,295,2132.07,7.227356
4,2010,230,1758.55,7.645870
5,2011,223,1799.15,8.067937
6,2012,296,2748.34,9.284932
7,2013,388,3634.40,9.367010
8,2014,1473,11604.65,7.878242
9,2015,2409,16670.33,6.920021
